# Python code for the Data wrangling Open Street Map project

By - Alexandre Medeiros Gonçalves

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import codecs
import json
from pymongo import MongoClient


### Producing a 50 times smaller file to test code (5.4Mb)

In [2]:
OSM_FILE = "rio-de-janeiro_brazil.osm"  

# Udacity lesson reference for creating a smaller copy of the file to test code. 
#The function writes lines in the new files for every multiple of 'times' lines read from the original file.

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def write_file(SAMPLE_FILE, times):
    with open(SAMPLE_FILE, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')

        # Write every 10th top level element
        for i, element in enumerate(get_element(OSM_FILE)):
            if i % times == 0:
                output.write(ET.tostring(element, encoding='utf-8'))

        output.write('</osm>')       
     

In [3]:
write_file("50_sample_rio-de-janeiro.osm" , 50)  

In [4]:
#Defining variable for treated file

OSM_FILE = "rio-de-janeiro_brazil.osm"

### Next we adapt the lesson quizes to have a quick overview of data

## QUIZ 1 - Iterative parsing - tag-number dictionary

In [5]:
#fill a dictionary cointaining all tags and correspondent number os appearences in the document

def count_tags(filename):
    tagcounts = {}
    for event, elem in ET.iterparse(filename):
        if event == 'end':
            if elem.tag not in tagcounts:
                tagcounts[elem.tag] = 1
            else:
                tagcounts[elem.tag] += 1
        elem.clear() # discard the element

    return tagcounts

tags = count_tags(OSM_FILE)
pprint.pprint(tags)
 
    


{'bounds': 1,
 'member': 24723,
 'nd': 1533886,
 'node': 1286043,
 'osm': 1,
 'relation': 3741,
 'tag': 474958,
 'way': 144776}


## Quiz 2 - tag types 

In [6]:
#patterns to check (lowercase, lowercase with 1 colon and problematic chars)
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#check patterns against the values of attibute k of tags

def key_type(element, keys):
    if element.tag == "tag":
        kvalue = element.get('k')
        if problemchars.search(kvalue) != None:
            keys['problemchars']+=1
        elif lower.search(kvalue)!= None:
            keys['lower']+= 1
        elif lower_colon.search(kvalue)!= None:
            keys['lower_colon']+= 1
        else:
            keys['other']+=1
        
        pass
        
    return keys

#defines the initial dictionary and iterates on the file (iterparse).

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


keys = process_map(OSM_FILE)
pprint.pprint(keys)



{'lower': 402696, 'lower_colon': 49089, 'other': 23173, 'problemchars': 0}


There are many other values and no problemchars. We investigate the first.

## Auditing 'other' k values - portuguese translated entries in uppercase

In [7]:
def show_category_other(filename,times): #prints every 'times' occurences
    i=0
    other = {}
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            kvalue = element.get('k')
            vvalue = element.get('v')
            if problemchars.search(kvalue) == None and lower.search(kvalue)== None and lower_colon.search(kvalue)== None:
                if i % times == 0:
                    other[kvalue] = vvalue
            i+=1
            pass
        
    return other

show_category_other(OSM_FILE,500)

{'IPP:BAIRRO': u'Maracan\xe3',
 'IPP:CHAVE': '11874',
 'IPP:CODATIVIDA': '0',
 'IPP:CODBAIRRO': '032',
 'IPP:CODIGO': '22988',
 'IPP:CODLINFERR': '1830',
 'IPP:COD_INEP': '33083070',
 'IPP:COD_QUADRA': '285A16454',
 'IPP:COD_SMA': '011494',
 'IPP:CRE': '05',
 'IPP:CodFavela': '294',
 'IPP:DESIGNACAO': '0918203',
 'IPP:ENDERECO': u'Rua Adail, 49\xa0\xa0',
 'IPP:ENDNOVO': 'http://webapp.sme.rio.rj.gov.br/jcartela/publico/pesquisa.do?idSetor=11163&cmd=load',
 'IPP:FLG_VALIDA': 'S',
 'IPP:OBS': u'Segundo a servente da UE as infoma\xe7\xf5es est\xe3o corretas.',
 'IPP:TELEFONES': '2413-4755',
 'parking:condition:left': 'free',
 'parking:condition:right': 'free',
 'parking:lane:both': 'no_stopping',
 'parking:lane:left': 'parallel',
 'parking:lanes:both': 'no_parking',
 'seamark:buoy_lateral:colour': 'red',
 'seamark:buoy_lateral:shape': 'pillar',
 'seamark:light:colour': 'red',
 'seamark:pipeline_submarine:category': 'oil'}

There are two main problems:
- two colon k types, that we will be dealt in the file transformation to json. 

- k-types translated to portuguese in uppercase.  

How big is this problem in the case of IPP:BAIRRO? (suburb)

In [8]:
count = 0
for _, element in ET.iterparse(OSM_FILE):
    for child in element:
        if child.get('k') == "IPP:BAIRRO":
            count += 1
print count        

1238


We chose to fix this translation back to english for two important examples: IPP:BAIRRO to addr:suburb  and  IPP:ENDERECO to addr:street. The last is not an exact match, as addresses in portuguese (and in this inputs) have the format "street,house number" so we need to get only what is before coma.

Here we chose to write another osm file to save changes, as we would like to audit street names next with the correct k values. We use the get_elements function and write a function very similar to write_file to rewrite the file with the proposed corrections only.

In [9]:
def write_file_translation_fixes(SAMPLE_FILE):
    with open(SAMPLE_FILE, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')

        for element in get_element(OSM_FILE):
            for child in element:

                if child.tag == "tag":
                    kvalue = child.get('k')
                    vvalue = child.get('v')

                    if kvalue == "IPP:BAIRRO":
                        child.set('k', 'addr:suburb') #translation

                    elif kvalue == "IPP:ENDERECO":
                        child.set('k', "addr:street")
                        child.set('v', vvalue.split(",")[0]) #In portuguese, full address (ENDERECO) 
                                                             # has the format street, house number 
                    pass
            output.write(ET.tostring(element, encoding='utf-8'))     
               
        output.write('</osm>') 

write_file_translation_fixes("rio-de-janeiro_brazil_kfixed.osm")

OSM_FILE = "rio-de-janeiro_brazil_kfixed.osm"  #updating the osm_file variable for further use

print "ok"

ok


In [10]:
print process_map(OSM_FILE)


{'problemchars': 0, 'lower': 402696, 'other': 21111, 'lower_colon': 51151}


We have fixed 2062 entries total.

## Quiz 3 - Exploring Users

How many unique users?

In [11]:
def process_map_uid(filename):
    
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('uid') != None:
            users.add(element.get('uid'))
        pass
    
    return users

users = process_map_uid(OSM_FILE)
len(users)
#pprint.pprint(users)
    

1562

## Quiz 4 - Auditing street name - brazilian standard

Next we use the code provided by lesson 6 to audit street types. The difference here is that types appear at the beginning of address strings.

In [12]:
#Changed pattern to get the first word (portugueses pattern) 
street_type_re = re.compile(r'\b\S+\.?', re.IGNORECASE)

#We need to use decode('unicode-escape') to deal with portuguese chars
expected = ["Rua", "Avenida", "Travessa", "Estrada", "Ladeira", 
            "Largo" , "Praia", "Via", "Alameda","Beco","Caminho",
            "Pra\xe7a".decode('unicode-escape'),"Rodovia"]




def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types




st_types = audit_street(OSM_FILE)
pprint.pprint(dict(st_types))


{'15': set(['15 de Novembro']),
 '199': set(['199']),
 'ALAMEDA': set(['ALAMEDA CALDENSE253']),
 'AVENIDA': set(['AVENIDA ANTARESS/N',
                 'AVENIDA BRASIL18476',
                 'AVENIDA DO CANALS/N',
                 'AVENIDA DO CONTORNO1',
                 'AVENIDA DO MAGISTERIOS/N',
                 'AVENIDA HENRIETTE DE HOLANDA AMADOS/N',
                 'AVENIDA ILHA DO FUNDAO107',
                 'AVENIDA JOAO XXIII222',
                 'AVENIDA JOAO XXIII230',
                 'AVENIDA VISCONDE DE NITEROI774']),
 'Adelaide': set(['Adelaide Badenes']),
 'Afredo': set(['Afredo Ceschiatti']),
 'Alfredo': set(['Alfredo Ceschiatti']),
 u'Apurin\xe3s': set([u'Apurin\xe3s']),
 u'Aterro': set([u'Aterro da Mar\xe9']),
 'Auto': set(['Auto Estrada Lagoa-Barra']),
 u'Av': set(['Av Castelo Branco',
             'Av Padre Anchieta',
             'Av Rotary',
             u'Av das Am\xe9ricas']),
 'Av.': set(['Av. Afranio de Melo Franco', 'Av. Brasil']),
 'Boulevard': set(['Bo

Fixing street types

In [13]:
#We need to use decode('unicode-escape') to deal with portuguese chars

mapping = { 
            "estrada" : "Estrada",
            "ESTRADA" : "Estrada",
            "Est." : "Estrada",
            "Estr." : "Estrada",
            "Trav" : "Travessa",
            "Trav." : "Travessa",
            "Rod." : "Rodovia",
            "Av": "Avenida",
            "Av.": "Avenida",
            "AVENIDA" : "Avenida",
            "Pca": "Pra\xe7a".decode('unicode-escape'),
            "P\xe7a.".decode('unicode-escape') : "Pra\xe7a".decode('unicode-escape'),
            "P\xe7a".decode('unicode-escape') : "Pra\xe7a".decode('unicode-escape'),
            "Praca": "Pra\xe7a".decode('unicode-escape'),
            "R." : "Rua",
            "RUA" : "Rua",
            "Rue" : "Rua",
            "rua" : "Rua",
            "Rod" : "Rodovia",
            "ALAMEDA" :"Alameda"
          }

def update_street_name(name, mapping):

    firstword = name.split()[0]
    name = name.replace(firstword, mapping[firstword])

    return name

for st_type, ways in st_types.iteritems():
        for name in ways:
            if name.split()[0] in mapping.keys():
                better_name = update_street_name(name, mapping)
                print name, "=>", better_name


Pça. Soldado Geraldo Cruz => Praça Soldado Geraldo Cruz
Pça. da República => Praça da República
Pça. da Bíblia => Praça da Bíblia
Pça. Gal. Alcio Souto => Praça Gal. Alcio Souto
Pça. José Alves de Azevedo => Praça José Alves de Azevedo
Pça. Belmonte => Praça Belmonte
Pça. dos Jesuítas => Praça dos Jesuítas
Pça. Ferreira de Abreu => Praça Ferreira de Abreu
Pça. Rosária Trotta => Praça Rosária Trotta
Pça. Laguna => Praça Laguna
Pça. Marcelino Gama => Praça Marcelino Gama
Pça. Estado de Israel => Praça Estado de Israel
Pça. dos Ucranianos => Praça dos Ucranianos
Pça. Santa Rosalia => Praça Santa Rosalia
Pça. Ricardo Gonçalves => Praça Ricardo Gonçalves
Pça. Irineu Machado => Praça Irineu Machado
Pça. Sentinela => Praça Sentinela
Pça. Mal. Hermes => Praça Mal. Hermes
Pça. Abuna => Praça Abuna
Pça. da Bandeira => Praça da Bandeira
Pça. Vicente de Oliveira Silva => Praça Vicente de Oliveira Silva
Pça. Emboaba => Praça Emboaba
Pça. Dr. José Pontes => Praça Dr. José Pontes
Pça. Confederação Su

## Auditing postcode - brazilian format

Postal code in Brazil has the following format  XXXXX-XXX, where X are integers form 0 to 9. Next we check entries that don't follow this pattern.

In [14]:
#postal code format regex
postcode_re = re.compile(r'^([0-9]{5})(-)([0-9]{3})$')

def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_post(osmfile):
    osm_file = open(osmfile, "r")
    post_types = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postcode(tag) and postcode_re.match(tag.attrib['v']) == None:
                    post_types.append(tag.attrib['v'])
                    
    osm_file.close()
    return post_types

unformatted_postal_codes = audit_post(OSM_FILE)
pprint.pprint(unformatted_postal_codes)


['22240004',
 '20270060',
 '25660004',
 '22280030',
 '22270010',
 '24346030',
 '2261001',
 '24324270',
 '22410000',
 '22281020',
 '20510180',
 '25010060',
 '26130130',
 '26130130',
 '26130130',
 '26130130',
 '26130130',
 '25.620-003',
 '20720010',
 '25953671',
 '25963082',
 '2695307',
 '26953201',
 '25955240',
 '25958060',
 '35953060',
 '24230',
 '23025 520',
 '20770240',
 '20775090',
 '20550200',
 '20550200',
 '24744520',
 '21921000',
 '22440040',
 '22440040',
 '22.776-070',
 '26130230',
 '26130230',
 '21311050',
 '20745000',
 '26900000',
 '20260200',
 '25940000',
 '20261130',
 '20520050',
 '8723CP',
 '8723CP',
 '4206XS',
 '4206XS',
 '24020000',
 '21941005',
 '22790790',
 '8426GK',
 '25900213',
 '25900213',
 '21515090',
 '20090-00',
 '22430090',
 '24220480',
 '24060037',
 '24060037',
 '24060037',
 '24060037',
 '21230354',
 '22631030',
 u'20770\u2011001',
 '22440033',
 '22750009',
 '20216005',
 '52645100',
 '2246-000',
 '22471340',
 '22471340',
 '22471340',
 '22471270',
 '22471340',
 '

There are a few variants, but the 8 digit sequence without hyphenation stands out. Next we fix this specific problem.

In [15]:
postcode_digit8_re = re.compile(r'^([0-9]{8})$')

def update_postcode(postalcode):
    name = postalcode[:5] + "-" + postalcode[5:]
    return name


for postalcode in unformatted_postal_codes:
    if postcode_digit8_re.match(postalcode) != None:
        better_name = update_postcode(postalcode)
        print postalcode, "=>", better_name
        


22240004 => 22240-004
20270060 => 20270-060
25660004 => 25660-004
22280030 => 22280-030
22270010 => 22270-010
24346030 => 24346-030
24324270 => 24324-270
22410000 => 22410-000
22281020 => 22281-020
20510180 => 20510-180
25010060 => 25010-060
26130130 => 26130-130
26130130 => 26130-130
26130130 => 26130-130
26130130 => 26130-130
26130130 => 26130-130
20720010 => 20720-010
25953671 => 25953-671
25963082 => 25963-082
26953201 => 26953-201
25955240 => 25955-240
25958060 => 25958-060
35953060 => 35953-060
20770240 => 20770-240
20775090 => 20775-090
20550200 => 20550-200
20550200 => 20550-200
24744520 => 24744-520
21921000 => 21921-000
22440040 => 22440-040
22440040 => 22440-040
26130230 => 26130-230
26130230 => 26130-230
21311050 => 21311-050
20745000 => 20745-000
26900000 => 26900-000
20260200 => 20260-200
25940000 => 25940-000
20261130 => 20261-130
20520050 => 20520-050
24020000 => 24020-000
21941005 => 21941-005
22790790 => 22790-790
25900213 => 25900-213
25900213 => 25900-213
21515090 =

## Deploying clean data to json file

In [16]:
# defining the created pattern of database
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

#main function
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way":
        node["type"] = element.tag     
        pos = [0,0]
        
        #dealing with element attributes first
        for attribute in element.attrib:  
            
            if attribute in CREATED: 
                if "created" not in node: node["created"] = {}  #create nested dictionary if not already created
                node["created"][attribute] = element.attrib[attribute]
                
            elif attribute == "lat":
                pos[0] = float(element.attrib[attribute]) #turning string to float
            
            elif attribute == "lon":
                pos[1] = float(element.attrib[attribute]) 
                
            else:
                node[attribute] = element.attrib[attribute]
       
        if pos != [0,0]: node['pos'] = [pos[0],pos[1]] #add position only when there is lat and long data
        
        #now the child elements and its attributes
        for child in element:
            
            if child.tag == "tag":          #elements named tag
                key = child.attrib['k']
                value = child.attrib['v']
                if problemchars.search(key):pass   #pass k problematic chars
                
                elif lower_colon.search(key):    #lowercase one colon
                    
                    #special treatment for addr:
                    if re.search("addr:",key ):  
                        
                        #fixing street names with update function
                        if is_street_name(child) and value.split()[0] in mapping.keys():
                            value = update_street_name(value,mapping)
                        
                        #fixing postal code with update function
                        elif is_postcode(child) and postcode_re.match(value) == None:
                            value = update_postcode(value)
                        pass 
                        
                        if "address" not in node: node["address"] = {}  #create subdictionary address if not present already
                        node["address"][key[5:]] = value
                    
                    #general treatment for one colon (nested dictionary)
                    else:
                        beginning = key.split(':')[0]
                        end = key.split(':')[1]
                        if beginning not in node: node[beginning] = {}
                        
                        elif not isinstance(node[beginning], dict):    #this happens when there is already a str value for
                            main_value = node[beginning]               #node[beginning], created by the lower.search(key) part
                            node[beginning] = {}                       #of the code below. We rebuild the structure putting 
                            node[beginning]["main_value"] = main_value #the previously created value in the main_value 
                            node[beginning][end] = value               #key of the dictionary
                        
                        else:
                            node[beginning][end] = value
                        
                elif lower.search(key):
                    node[key] = value
        
            if child.tag == "nd":
                if "node_refs" not in node: node["node_refs"] = []
                node["node_refs"].append(child.attrib["ref"])    
         
        return node
    else:
        return None


def process_map_json(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


process_map_json(OSM_FILE,pretty = False)
print "ok"


ok


## Making MongoDB queries to JSON file

In [17]:
#To load the json file, start the mongoDbserver
#mongod --dbpath dataPATH
#and run
#mongoimport --db rio --collection rio --drop --file `PATH`\rio-de-janeiro_brazil_kfixed.json

client = MongoClient('mongodb://localhost:27017')

db = client.rio

db.rio.find().count()

# function to loop through result objects returned by pymongo queries
def show(cursor):
    for document in cursor:
        pprint.pprint(document)
    return
  

### Number of unique users and top 10 contributors

In [18]:
len(db.rio.distinct("created.user"))


1442

In [19]:
cursor = db.rio.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])
show(cursor)

{u'_id': u'Alexandrecw', u'count': 377657}
{u'_id': u'AlNo', u'count': 174518}
{u'_id': u'ThiagoPv', u'count': 123929}
{u'_id': u'Import Rio', u'count': 87800}
{u'_id': u'Geaquinto', u'count': 74631}
{u'_id': u'Nighto', u'count': 66948}
{u'_id': u'Thundercel', u'count': 60824}
{u'_id': u'M\xe1rcio V\xedn\xedcius Pinheiro', u'count': 32014}
{u'_id': u'Ricardo Mitidieri', u'count': 26189}
{u'_id': u'patodiez', u'count': 25399}


### Number of nodes and ways

In [20]:
db.rio.find({"type":"way"}).count()

144695

In [21]:
db.rio.find({"type":"node"}).count()

1285924

### Postal code count

In [22]:
cursor = db.rio.aggregate([{"$group":{"_id":"$address.postcode", "count":{"$sum":1}}}, 
                   {"$sort":{"count":-1}},
                   {"$limit" : 6}])
show(cursor)

{u'_id': None, u'count': 1428391}
{u'_id': u'22720-410', u'count': 96}
{u'_id': u'22471-003', u'count': 84}
{u'_id': u'22720-400', u'count': 76}
{u'_id': u'22220-000', u'count': 62}
{u'_id': u'22221-000', u'count': 56}


### Suburb entries count - Tijuca

In [23]:
cursor = db.rio.aggregate([{"$group":{"_id":"$address.suburb", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])

show(cursor)

{u'_id': None, u'count': 1428539}
{u'_id': u'Centro', u'count': 127}
{u'_id': u'Campo Grande', u'count': 102}
{u'_id': u'Recreio dos Bandeirantes', u'count': 100}
{u'_id': u'Ipanema', u'count': 90}
{u'_id': u'Santa Cruz', u'count': 80}
{u'_id': u'Bangu', u'count': 72}
{u'_id': u'Barra da Tijuca', u'count': 69}
{u'_id': u'Tijuca', u'count': 48}
{u'_id': u'Botafogo', u'count': 48}


In [24]:
cursor = db.rio.aggregate([{"$match":{"amenity":"restaurant"}},
                           {"$match":{"address.suburb" : "Tijuca"}},
                   {"$group":{"_id":"$cuisine", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":5}])

show(cursor)

{u'_id': u'seafood', u'count': 1}
{u'_id': u'italian', u'count': 1}


### Amenity and cuisine counts

In [25]:
cursor = db.rio.aggregate([{"$group":{"_id":"$amenity",
"count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])

show(cursor)

{u'_id': None, u'count': 1419560}
{u'_id': u'school', u'count': 1759}
{u'_id': u'bicycle_parking', u'count': 1139}
{u'_id': u'restaurant', u'count': 940}
{u'_id': u'parking', u'count': 893}
{u'_id': u'fast_food', u'count': 849}
{u'_id': u'fuel', u'count': 614}
{u'_id': u'place_of_worship', u'count': 508}
{u'_id': u'bank', u'count': 489}
{u'_id': u'pub', u'count': 379}


In [26]:
cursor = db.rio.aggregate([{"$match":{"amenity":"restaurant"}},
                           {"$group":{"_id":"$cuisine", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":6}])

show(cursor)

{u'_id': None, u'count': 563}
{u'_id': u'pizza', u'count': 85}
{u'_id': u'regional', u'count': 81}
{u'_id': u'italian', u'count': 35}
{u'_id': u'japanese', u'count': 34}
{u'_id': u'steak_house', u'count': 18}
